In [10]:
import geemap
import numpy as np
import ee

In [11]:
Map= geemap.Map(center=[40,-100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [12]:
#study area
region_shp= '/Users/allen/OneDrive - University of Oklahoma/CRESTHH/Case study - Harvey/Houston_basin.shp'
region= geemap.shp_to_ee(region_shp)
Map.addLayer(region)

# Acquisition of before before-event tile and after-event tile

Smooth them with focal mean

In [13]:
SAR_before= ee.Image('COPERNICUS/S1_GRD/S1B_IW_GRDH_1SDV_20170818T122205_20170818T122235_006994_00C525_B566').clip(region)
before_filtered = SAR_before.select('VH').focal_mean(50, 'circle', 'meters')
Map.addLayer(before_filtered, {'min':-30, 'max':5}, 'before event')

In [14]:
SAR_after=ee.Image('COPERNICUS/S1_GRD/S1B_IW_GRDH_1SDV_20170830T122203_20170830T122232_007169_00CA2C_E7BF').clip(region)
after_filtered = SAR_after.select('VH').focal_mean(50, 'circle', 'meters')
Map.addLayer(after_filtered.select('VH'), {'min':-30, 'max':5}, 'after event')

# Calculate the difference between before-event and after-event

In [15]:
threshold= 1.25 # to adjust the threshold
difference = after_filtered.divide(before_filtered)
difference_binary = difference.gt(threshold)

# Further refine the results with global open water dataset and DEM

In [16]:
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
swater_mask = swater.eq(10).updateMask(swater.gte(10)) #water presents larger than 10 months are regarded as permenant
flooded_mask = difference_binary.where(swater_mask,0)
flooded = flooded_mask.updateMask(flooded_mask)
connections = flooded.connectedPixelCount() # filter out single pixels
flooded = flooded.updateMask(connections.gte(8))

# filter out places with slope larger than 5 percent
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM') #probably we can use HAND index
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
flooded = flooded.updateMask(slope.lt(5))

In [17]:
Map.addLayer(flooded,{'palette': 'blue'},'flooded')

In [22]:
Map = geemap.Map()
Map.addLayer(region)
Map.addLayer(flooded,{'palette': 'blue'},'flooded')
left_layer = geemap.ee_tile_layer(before_filtered, {'min':-30, 'max':1}, 'before Harvey')
right_layer = geemap.ee_tile_layer(after_filtered, {'min':-30, 'max':1}, 'after Harvey')
Map.split_map(left_layer,right_layer)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…